In [169]:
import pandas as pd
import numpy as np
import datetime as dt
import glob
from tqdm import tqdm
import os
pd.set_option("display.max_rows", 20)

### Load drying regimes data and non-perennial gages from PNW

In [37]:
dryReg = pd.read_csv("../data/dryingRegimes_data_RF.csv")
pnwNP = pd.read_csv("../data/pnwNP_Info.csv")

### Make columns in "YYYY-MM-DD" format for each drying event

In [25]:
dryReg["dryStart"] = pd.to_datetime(dryReg["calendar_year"], format='%Y')+ pd.to_timedelta(dryReg["dry_date_start"], unit='days')
dryReg["dryEnd"] = dryReg["dryStart"]+ pd.to_timedelta(dryReg["dry_dur"], unit='days')
dryReg["dryPeakDate"] = pd.to_datetime(dryReg["calendar_year"], format='%Y')+ pd.to_timedelta(dryReg["peak_date"], unit='days')

### Subset drying regimes data to PNW gages

In [27]:
pnwNPdry = dryReg[dryReg["gage"].isin(pnwNP['gage'])].reset_index()

### Load, subset to drying events, and save the modeled data

In [173]:
path =r'../data/pnwNP_modeledData/'

for i in tqdm(range(90,110)):
    filename = glob.glob(path + str(pnwNPdry['gage'][i])+"*wNWM2d1.csv")
    
    df = pd.read_csv(filename[0]).drop(["Unnamed: 0","Unnamed: 0.1","gage_x","gage_y"],axis=1)
    df["time"] = pd.to_datetime(df["time"])
    
    df = df[(df['time'] >= pnwNPdry["dryPeakDate"][i])&(df['time'] <= pnwNPdry["dryEnd"][i])]
    df['state'] = 0
    df.loc[(df['time'] >= pnwNPdry["dryPeakDate"][i])&(df['time'] < pnwNPdry["dryStart"][i]),'state'] = "drying"
    df.loc[(df['time'] >= pnwNPdry["dryStart"][i])&(df['time'] <= pnwNPdry["dryEnd"][i]),'state'] = "dry"
    
    
    if os.path.isfile(str(pnwNPdry['gage'][i])+"dryingEvent.csv"):
        df.to_csv("../data/pnwDrying_model/" + str(pnwNPdry['gage'][i])+"dryingEvent.csv", mode='a', index=False, header=None)
    else:
        df.to_csv("../data/pnwDrying_model/" + str(pnwNPdry['gage'][i])+"dryingEvent.csv", index=False)

100%|██████████| 20/20 [00:00<00:00, 30.81it/s]
